In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

# tensorboard --logdir=./logs --port=any number you want
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

C:\Users\joobong\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# one hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    # one cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[ 0.37915292, -0.54918814]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequances') as scope:
    # one cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.5819289 ,  0.43208978],
        [ 0.88217247,  0.8154548 ],
        [ 0.13342865, -0.631108  ],
        [ 0.27409574, -0.78612137],
        [ 0.2372579 ,  0.22885989]]], dtype=float32)


In [5]:
with tf.variable_scope('3_batches') as scope:
    # one cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.02660759,  0.11081155],
        [ 0.04162992,  0.24706486],
        [ 0.13791427,  0.12174579],
        [ 0.19353995,  0.03425804],
        [ 0.09056399,  0.01147817]],

       [[ 0.04824106,  0.13433287],
        [ 0.02340621,  0.04905285],
        [ 0.09890956, -0.00628696],
        [ 0.1509757 , -0.0571177 ],
        [ 0.1873079 , -0.09025289]],

       [[ 0.07045752, -0.05650955],
        [ 0.12282095, -0.09271558],
        [ 0.11807364,  0.07335638],
        [ 0.15591517,  0.1952341 ],
        [ 0.24918218,  0.08639618]]], dtype=float32)


In [6]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # one cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.09501679,  0.14972918],
        [-0.03194668,  0.12671547],
        [-0.12083457,  0.04190237],
        [-0.16853432, -0.00419868],
        [ 0.03390745,  0.05028253]],

       [[-0.06904674,  0.0414968 ],
        [ 0.10306758,  0.1010014 ],
        [-0.05347221,  0.01525201],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.10068073, -0.02370807],
        [-0.15261619, -0.04099264],
        [-0.1699811 , -0.00047414],
        [-0.16270062,  0.03405712],
        [ 0.        ,  0.        ]]], dtype=float32)


In [7]:
with tf.variable_scope('initial_state') as scope:    
    # one cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    
    batch_size = 3
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.08097942, -0.03344389],
        [ 0.06578816, -0.03407852],
        [-0.01006975,  0.15560019],
        [-0.09605072,  0.27504328],
        [-0.01435238,  0.00877616]],

       [[-0.02190757,  0.00456008],
        [ 0.03228641, -0.0814694 ],
        [-0.03925359,  0.084735  ],
        [-0.11598061,  0.2433754 ],
        [-0.18286566,  0.33667213]],

       [[-0.07919138,  0.1919204 ],
        [-0.15284446,  0.30400854],
        [-0.14013268,  0.17600645],
        [-0.14629571,  0.13513927],
        [-0.22286975,  0.3091094 ]]], dtype=float32)


In [8]:
# create input data
batch_size = 3
sequence_length = 5
input_dim = 3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [9]:
with tf.variable_scope('generated_data') as scope:
    # one cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[-1.6153963e-01, -2.5056832e-02,  8.4046140e-02, -7.4945979e-02,
         -1.4603852e-01],
        [-1.4972551e-01, -2.6785075e-03,  5.9117419e-01,  8.3772212e-02,
         -1.5623972e-01],
        [-3.6169995e-02, -1.6148151e-04,  7.4822354e-01,  1.0547684e-01,
         -5.9734717e-02],
        [-8.8989474e-03, -7.1327695e-06,  7.6024175e-01,  5.1688589e-02,
         -2.0005183e-02],
        [-2.2156690e-03, -2.8343044e-07,  7.6138109e-01,  2.4470503e-02,
         -7.1690660e-03]],

       [[-5.7257019e-04, -1.4645556e-11,  7.6143450e-01,  1.3774626e-02,
         -3.6466031e-03],
        [-1.2483087e-04, -9.7961873e-13,  7.6158792e-01,  5.1992475e-03,
         -9.6022291e-04],
        [-2.9173618e-05, -3.7449675e-14,  7.6159292e-01,  2.3706360e-03,
         -3.5225594e-04],
        [-6.7873775e-06, -1.3207296e-15,  7.6159394e-01,  1.0783280e-03,
         -1.2939572e-04],
        [-1.5765870e-06, -4.4106197e-17,  7.6159412e-01,  4.9000635e-04,
         -4.7541962e-05]],

      